# Financial Inclusion in Africa


Financial inclusion remains one of the main obstacles to economic and human development in Africa. For example, across Kenya, Rwanda, Tanzania, and Uganda only 9.1 million adults (or 14% of adults) have access to or use a commercial bank account.

## Objectives

The objective of this competition is to create a machine learning model to predict which individuals are most likely to have or use a bank account. The models and solutions developed can provide an indication of the state of financial inclusion in Kenya, Rwanda, Tanzania and Uganda, while providing insights into some of the key factors driving individuals’ financial security.

## Data set

We are asked to predict the likelihood of the person having a bank account or not (Yes = 1, No = 0), for each unique id in the test dataset . We will train our model on 70% of the data and test your model on the final 30% of the data, across four East African countries - Kenya, Rwanda, Tanzania, and Uganda.

The main dataset contains demographic information and what financial services are used by approximately 33,600 individuals across East Africa. This data was extracted from various Finscope surveys ranging from 2016 to 2018, and more information about these surveys can be found here:

### Country	
Country interviewee is in.
### Year	
Year survey was done in.
### Uniqueid	
Unique identifier for each interviewee
### Location_type	
Type of location: Rural, Urban
### Cellphone_access	
If interviewee has access to a cellphone: Yes, No
### Household_size	
Number of people living in one house
### Age_of_respondent	
The age of the interviewee
### Gender_of_respondent	
Gender of interviewee: Male, Female
### Relationship_with_head	
The interviewee’s relationship with the head of the house:Head of Household, Spouse, Child, Parent, Other relative, Other non-relatives, Dont know
### Marital_status	
The martial status of the interviewee: Married/Living together, Divorced/Seperated, Widowed, Single/Never Married, Don’t know
### Education_level	
Highest level of education: No formal education, Primary education, Secondary education, Vocational/Specialised training, Tertiary education, Other/Dont know/RTA
### Job_type	
Type of job interviewee has: Farming and Fishing, Self employed, Formally employed Government, Formally employed Private, Informally employed, Remittance Dependent, Government Dependent, Other Income, No Income, Dont Know/Refuse to answer


## Approach

### Importing the dataset

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as stats


In [145]:
df = pd.read_csv("./train.csv")
# inspecting the dataset 
df.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [146]:
df.nunique()

country                      4
year                         3
uniqueid                  8735
bank_account                 2
location_type                2
cellphone_access             2
household_size              20
age_of_respondent           85
gender_of_respondent         2
relationship_with_head       6
marital_status               5
education_level              6
job_type                    10
dtype: int64

In [147]:
df.dtypes

country                   object
year                       int64
uniqueid                  object
bank_account              object
location_type             object
cellphone_access          object
household_size             int64
age_of_respondent          int64
gender_of_respondent      object
relationship_with_head    object
marital_status            object
education_level           object
job_type                  object
dtype: object

### Checking for Null Data

In [148]:
df.isna().sum()

country                   0
year                      0
uniqueid                  0
bank_account              0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
dtype: int64

### Checking for duplicate data

In [149]:
df.duplicated().sum()

0

### Removing unnecessary data

In [150]:
df.drop("uniqueid",axis=1,inplace=True)

### Checking for correlation between variables

In order to use correlation between variables we need to implement one hot encoding on categorical data so that they can be used for analysis too

In [151]:
# One Hot Encoding
dummy = pd.get_dummies(df.select_dtypes(include="object"),drop_first=True)
dummy = dummy.astype(int)
list1 = [x for x in list(dummy.columns) if "bank" in x]
list1

['bank_account_Yes']

In [152]:
df = pd.concat([dummy,df.select_dtypes(exclude=object)],axis=1)
# Returing bank_account to original name 
df =df.rename({"bank_account_Yes":"bank_account"},axis=1)


In [153]:
# df.corr().style.background_gradient("coolwarm")

### Starting Analysis by using Linear Regression

#### Define our dependant and independant variable

In [154]:
Y_axis = df["bank_account"]
X_axis = stats.add_constant(df.drop(["bank_account"],axis=1))

#### Create our model

In [155]:
model = stats.OLS(Y_axis,X_axis)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           bank_account   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     271.2
Date:                Thu, 29 Feb 2024   Prob (F-statistic):               0.00
Time:                        06:47:00   Log-Likelihood:                -4936.9
No. Observations:               23524   AIC:                             9938.
Df Residuals:                   23492   BIC:                         1.020e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
const                                              -0.0001   7.32e-06    -15.210      0.000      -0.000   -9.69e-05
country_Rwanda                                     -0.0464      0.005     -8.592      0.000      -0.057      -0.036
country_Tanzania                                   -0.1317      0.007    -18.503      0.000      -0.146      -0.118
country_Uganda                                     -0.1611      0.009    -17.739      0.000      -0.179      -0.143
location_type_Urban                                 0.0422      0.005      8.798      0.000       0.033       0.052
cellphone_access_Yes                                0.0736      0.005     15.156      0.000       0.064       0.083
gender_of_respondent_Male                           0.0382      0.005      7.290      0.000       0.028       0.049
relationship_with_head_Head of Household            0.0770      0.009      8.725      0.000       0.060       0.094
relationship_with_head_Other non-relatives         -0.0142      0.023     -0.621      0.535      -0.059       0.031
relationship_with_head_Other relative               0.0017      0.014      0.128      0.898      -0.025       0.028
relationship_with_head_Parent                       0.0179      0.012      1.501      0.133      -0.005       0.041
relationship_with_head_Spouse                       0.0635      0.010      6.683      0.000       0.045       0.082
marital_status_Dont know                            0.0399      0.106      0.377      0.706      -0.168       0.248
marital_status_Married/Living together              0.0286      0.009      3.314      0.001       0.012       0.046
marital_status_Single/Never Married                 0.0047      0.008      0.607      0.544      -0.011       0.020
marital_status_Widowed                              0.0089      0.010      0.883      0.377      -0.011       0.029
education_level_Other/Dont know/RTA                 0.2177      0.051      4.288      0.000       0.118       0.317
education_level_Primary education                   0.0477      0.006      8.512      0.000       0.037       0.059
education_level_Secondary education                 0.1630      0.007     22.175      0.000       0.149       0.177
education_level_Tertiary education                  0.3900      0.011     35.849      0.000       0.369       0.411
education_level_Vocational/Specialised training     0.3841      0.012     31.043      0.000       0.360       0.408
job_type_Farming and Fishing                       -0.0657      0.027     -2.406      0.016      -0.119      -0.012
job_type_Formally employed Government               0.3326      0.031     10.631      0.000       0.271       0.394
job_type_Formally employed Private                  0.2314      0.0

In [157]:
print('The sum of square residuals is {:.1f}'.format(result.ssr))

The sum of square residuals is 2095.7


Due to the fact that the R squared and Adj R squared value are low we will try to remove some columns with pvalues > 0.05